928. Minimize Malware Spread II
Hard
Topics
premium lock icon
Companies
You are given a network of n nodes represented as an n x n adjacency matrix graph, where the ith node is directly connected to the jth node if graph[i][j] == 1.

Some nodes initial are initially infected by malware. Whenever two nodes are directly connected, and at least one of those two nodes is infected by malware, both nodes will be infected by malware. This spread of malware will continue until no more nodes can be infected in this manner.

Suppose M(initial) is the final number of nodes infected with malware in the entire network after the spread of malware stops.

We will remove exactly one node from initial, completely removing it and any connections from this node to any other node.

Return the node that, if removed, would minimize M(initial). If multiple nodes could be removed to minimize M(initial), return such a node with the smallest index.

 

Example 1:

Input: graph = [[1,1,0],[1,1,0],[0,0,1]], initial = [0,1]
Output: 0
Example 2:

Input: graph = [[1,1,0],[1,1,1],[0,1,1]], initial = [0,1]
Output: 1
Example 3:

Input: graph = [[1,1,0,0],[1,1,1,0],[0,1,1,1],[0,0,1,1]], initial = [0,1]
Output: 1
 

Constraints:

n == graph.length
n == graph[i].length
2 <= n <= 300
graph[i][j] is 0 or 1.
graph[i][j] == graph[j][i]
graph[i][i] == 1
1 <= initial.length < n
0 <= initial[i] <= n - 1
All the integers in initial are unique.

In [ ]:
class Solution:
    def minMalwareSpread(self, graph: List[List[int]], initial: List[int]) -> int:
        N = len(graph)
        clean = set(range(N)) - set(initial)
        def dfs(u, seen):
            for v, adj in enumerate(graph[u]):
                if adj and v in clean and v not in seen:
                    seen.add(v)
                    dfs(v, seen)

        # For each node u in initial, dfs to find
        # 'seen': all nodes not in initial that it can reach.
        infected_by = {v: [] for v in clean}
        for u in initial:
            seen = set()
            dfs(u, seen)

            # For each node v that was seen, u infects v.
            for v in seen:
                infected_by[v].append(u)

        # For each node u in initial, for every v not in initial
        # that is uniquely infected by u, add 1 to the contribution for u.
        contribution = collections.Counter()
        for v, neighbors in infected_by.items():
            if len(neighbors) == 1:
                contribution[neighbors[0]] += 1

        # Take the best answer.
        best = (-1, min(initial))
        for u, score in contribution.items():
            if score > best[0] or score == best[0] and u < best[1]:
                best = score, u
        return best[1]

In [ ]:
class UnionFind:
    def __init__(self, n):
        self.parent = list(range(n))
        self.size = [1] * n

    def find(self, x):
        if x != self.parent[x]:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        a, b = self.find(x), self.find(y)
        if a != b:
            if self.size[a] < self.size[b]:
                self.parent[a] = b
                self.size[b] += self.size[a]
            else:
                self.parent[b] = a
                self.size[a] += self.size[b]

class Solution:
    def minMalwareSpread(self, graph, initial):
        n, ans, max_save = len(graph), min(initial), 0
        infected = [False] * n
        for u in initial:
            infected[u] = True
        uf = UnionFind(n)
        
        # Union all non-infected nodes
        for i in range(n):
            if not infected[i]:
                for j in range(i + 1, n):
                    if not infected[j] and graph[i][j] == 1:
                        uf.union(i, j)
        
        # Count the influence of each initially infected node
        component = [set() for _ in range(n)]
        count = [0] * n
        for u in initial:
            component[u] = {uf.find(v) for v in range(n) if not infected[v] and graph[u][v] == 1}
            for v in component[u]:
                count[v] += 1
        
        # Determine the node removal that saves the most nodes
        for u in sorted(initial):
            save = sum(uf.size[v] for v in component[u] if count[v] == 1)
            if save > max_save or (save == max_save and u < ans):
                max_save, ans = save, u
        
        return ans